# Query AMO with Add-on GUID

In Telemetry and elsewhere we typically use add-on GUIDs to uniquely represent specific add-ons. Often times a GUID is ambiguous, revealing little to no information about the add-on. This script allows a user to quickly get add-on names, e10s compatibility, versions, weekly downloads, categories, etc. from AMO with just an add-on GUID. See the Appendix for an example JSON blob displaying all possible fields. Aside from easily acquiring meta data for add-ons, this example shows the various fields the user can access *not* accessible via telemetry at the moment.

The example below is a simplification of the [script](https://github.com/andymckay/new-arewee10syet.com/blob/master/build.py) used to generate the [arewee10syet.com](https://arewee10syet.com) page. For more details please see the [AMO API doc](https://addons-server.readthedocs.io/en/latest/topics/api/addons.html).


In [ ]:
import pandas as pd
import os
import requests
import json
import urllib
import sys

# Set Up


In [ ]:
# for manual editting of missing or incorrect add-on names
fixups = {
    'testpilot@labs.mozilla.com': 'Test Pilot (old one)',
    '{20a82645-c095-46ed-80e3-08825760534b}': 'Microsoft .NET framework assistant',
}

def process_amo(result):
    """
    Selects and processes specific fields from the dict,
    result, and returns new dict
    """
    try:
        name = result['name']['en-US']
    except KeyError:
        name = result['slug']
    return {
        'name': name,
        'url': result['url'],
        'guid': result['guid'],
        'e10s_status': result['e10s'],
        'avg_daily_users': result['average_daily_users'],
        'categories': ','.join(result['categories']['firefox']),
        'weekly_downloads': result['weekly_downloads'],
        'ratings': result['ratings']
    }

def amo(guid, raw=False):
    """
    Make AMO API call to request data for a given add-on guid 
    
    Return raw data if raw=True, which returns the full
    json returned from AMO as a python dict, otherwise call 
    process_amo() to only return fields of interest 
    (specified in process_amo())
    """
    addon_url = AMO_SERVER + '/api/v3/addons/addon/{}/'.format(guid)
    compat_url = AMO_SERVER + '/api/v3/addons/addon/{}/feature_compatibility/'.format(guid)

    result = {}
    print "Fetching Data for:", guid
    for url in (addon_url, compat_url):
        res = requests.get(url)
        if res.status_code != 200:
            return {
                'name': fixups.get(
                    guid, '{} error fetching data from AMO'.format(res.status_code)),
                'guid': guid
            }
        res.raise_for_status()
        res_json = res.json()
        result.update(res_json)
    if raw:
        return result
    return process_amo(result)

def reorder_list(lst, move_to_front):
    """
    Reorganizes the list <lst> such that the elements in
    <move_to_front> appear at the beginning, in the order they appear in
    <move_to_front>, returning a new list
    """
    result = lst[:]
    for elem in move_to_front[::-1]:
        assert elem in lst, "'{}' is not in the list".format(elem)
        result = [result.pop(result.index(elem))] + result
    return result
        

Instantiate amo server object to be used by the above functions

In [ ]:
AMO_SERVER = os.getenv('AMO_SERVER', 'https://addons.mozilla.org')

# Example: Request Data for 10 add-on GUIDs

As an example, we can call the `amo()` function for a list of 10 add-on GUIDs formatting them into a pandas DF.

In [ ]:
addon_guids = \
['easyscreenshot@mozillaonline.com',
 'firebug@software.joehewitt.com',
 'firefox@ghostery.com',
 'uBlock0@raymondhill.net',
 '{20a82645-c095-46ed-80e3-08825760534b}',
 '{73a6fe31-595d-460b-a920-fcc0f8843232}',
 '{DDC359D1-844A-42a7-9AA1-88A850A938A8}',
 '{b9db16a4-6edc-47ec-a1f4-b86292ed211d}',
 '{d10d0bf8-f5b5-c8b4-a8b2-2b9879e08c5d}',
 '{e4a8a97b-f2ed-450b-b12d-ee082ba24781}']

df = pd.DataFrame([amo(i) for i in addon_guids])

# move guid and name to front of DF
df = df[reorder_list(list(df), move_to_front=['guid', 'name'])]
df

There you have it! Please look at the Appendix for the possible fields obtainable through AMO.

# Appendix

The function `process_amo()` uses prespecified fields. Here you can take a look at a number of the available fields and make necessary edits.

In [ ]:
# request data for a single add-on guid
result = amo(addon_guids[1], raw=True)

In [ ]:
result